In [1]:
import os

# Función para instalar OpenJDK 8 si no está instalado previamente.
def install_java():
    # Verifica si la ruta del JDK existe.
    if not os.path.exists('/usr/lib/jvm/java-8-openjdk-amd64'):
        print("Instalando OpenJDK 8...")
        # Comando para instalar Java en modo silencioso.
        !apt-get install openjdk-8-jdk-headless -qq > /dev/null
        print("OpenJDK 8 instalado correctamente.")
    else:
        print("OpenJDK 8 ya está instalado.")



# Función para descargar Apache Spark solo si no está ya descargado.
def download_spark():
    # URL y nombre del archivo comprimido de Spark.
    spark_url = "https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si el archivo comprimido ya existe.
    if not os.path.exists(spark_tar):
        print("Descargando Apache Spark...")
        # Comando para descargar el archivo desde la URL.
        !wget -q $spark_url
        print("Descarga completa.")
    else:
        print("El archivo de Apache Spark ya está descargado.")



# Función para descomprimir el archivo de Apache Spark solo si no está descomprimido.
def extract_spark():
    # Nombre de la carpeta descomprimida.
    spark_dir = "spark-3.4.3-bin-hadoop3"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si la carpeta descomprimida ya existe.
    if not os.path.exists(spark_dir):
        print("Descomprimiendo Apache Spark...")
        # Comando para descomprimir el archivo.
        !tar xf $spark_tar
        print("Apache Spark descomprimido.")
    else:
        print("La carpeta de Apache Spark ya existe.")



# Función para configurar las variables de entorno necesarias para Spark.
def set_environment_variables():
    # Establece la ruta de JAVA_HOME y SPARK_HOME.
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"
    print("Variables de entorno configuradas.")



# Función para verificar si un paquete de Python ya está instalado.
import subprocess
import sys

def is_package_installed(package_name):
    try:
        # Ejecuta el comando `pip show` para verificar si el paquete está instalado.
        subprocess.check_call(
            [sys.executable, "-m", "pip", "show", package_name],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except subprocess.CalledProcessError:
        return False



# Función para instalar librerías de Python necesarias (findspark y pyspark).
def install_python_libraries():
    # Verifica e instala findspark.
    if not is_package_installed("findspark"):
        print("Instalando findspark...")
        !pip install -q findspark
    else:
        print("findspark ya está instalado.")

    # Verifica e instala pyspark.
    if not is_package_installed("pyspark"):
        print("Instalando pyspark...")
        !pip install -q pyspark
    else:
        print("pyspark ya está instalado.")



install_java()                # Instala OpenJDK 8
download_spark()              # Descarga Apache Spark
extract_spark()               # Descomprime Apache Spark
set_environment_variables()   # Configura las variables de entorno
install_python_libraries()    # Instala las librerías de Python

Instalando OpenJDK 8...
OpenJDK 8 instalado correctamente.
Descargando Apache Spark...
Descarga completa.
Descomprimiendo Apache Spark...
Apache Spark descomprimido.
Variables de entorno configuradas.
Instalando findspark...
pyspark ya está instalado.


---

### **¿Qué es el almacenamiento en caché en PySpark?**

En PySpark, el almacenamiento en caché (caching) permite guardar datos en la memoria para que puedan ser reutilizados en operaciones posteriores sin necesidad de recalcularlos o volver a leerlos desde el almacenamiento original. Esto mejora significativamente el rendimiento, especialmente cuando se realizan múltiples operaciones sobre los mismos datos.

---

### **¿Por qué usar caché?**
1. **Evita cálculos repetidos:** Cuando trabajas con transformaciones como `filter`, `map`, o `join`, Spark recalcula todo el conjunto de datos desde el principio si no está en caché.
2. **Acelera las operaciones:** Los datos almacenados en memoria se acceden mucho más rápido que los almacenados en disco o recomputados.
3. **Ideal para reutilización:** Si necesitas ejecutar múltiples acciones sobre el mismo DataFrame o RDD, caché es esencial.

---

### **¿Cómo usar el caché en PySpark?**
Puedes almacenar en caché un **RDD** o un **DataFrame** usando el método `cache()` o `persist()`.

1. **`cache()`:** Guarda los datos en memoria (por defecto). Ejemplo:
   ```python
   df.cache()  # DataFrame estará en caché
   df.show()   # Primera acción, los datos son cargados en memoria
   ```

2. **`persist(storageLevel)`:** Ofrece más control sobre cómo y dónde guardar los datos (por ejemplo, en disco o memoria). Ejemplo:
   ```python
   from pyspark import StorageLevel

   df.persist(StorageLevel.MEMORY_AND_DISK)  # Guarda en memoria y disco
   df.count()  # Primera acción, los datos se almacenan
   ```

---

### **Algunos niveles de persistencia comunes:**
- **`MEMORY_ONLY` (predeterminado):** Almacena solo en memoria. Si no cabe, los datos no se guardan.
- **`MEMORY_AND_DISK`:** Si no hay suficiente memoria, guarda el exceso en disco.
- **`DISK_ONLY`:** Guarda únicamente en disco (más lento).

---

### **Consideraciones:**
- Usa caché solo si los datos serán reutilizados. De lo contrario, ocupa memoria innecesariamente.
- Recuerda liberar la memoria con `unpersist()` cuando ya no necesites los datos:
  ```python
  df.unpersist()
  ```

---
---

# **Crear una sesion en Spark**

In [3]:
# Importar la biblioteca findspark, que ayuda a configurar PySpark correctamente en el entorno de ejecución
import findspark

# Inicializar findspark para establecer las variables necesarias de PySpark
findspark.init()

# Importar SparkSession desde la biblioteca pyspark.sql
# SparkSession es la entrada principal para trabajar con DataFrames en PySpark
from pyspark.sql import SparkSession

# Crear o obtener una SparkSession
# Esto inicializa un entorno de Spark si no existe uno, o reutiliza uno existente
spark = SparkSession.builder.getOrCreate()

# Obtener el contexto de Spark (SparkContext) desde la SparkSession
# SparkContext es la entrada principal para trabajar con RDDs en PySpark
sc = spark.sparkContext

In [19]:
# Crear un RDD paralelo con los números del 0 al 9
rdd = sc.parallelize([item for item in range(10)])
# `sc.parallelize` distribuye la colección `[0, 1, 2, ..., 9]` en las particiones del clúster.
# Resultado esperado: un RDD con los números del 0 al 9.
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [12]:
from pyspark.storagelevel import StorageLevel
# Importa el módulo `StorageLevel`, que define los niveles de persistencia para los RDD.

In [13]:
# Persistir el RDD en la memoria únicamente (MEMORY_ONLY)
rdd.persist(StorageLevel.MEMORY_ONLY)
# Guarda el RDD en la memoria RAM. Si la memoria es insuficiente, el resto de los datos no se guardará.
# No se realiza ninguna acción aquí, solo se indica a Spark que lo persista cuando sea necesario.
# Resultado esperado: Los datos estarán en memoria una vez que el RDD sea calculado.

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287

In [14]:
# Eliminar el RDD persistido de la memoria
rdd.unpersist()
# Libera el espacio en memoria donde estaba almacenado el RDD persistido.
# Resultado esperado: La memoria utilizada por este RDD queda liberada.

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287

In [15]:
# Persistir el RDD en disco únicamente (DISK_ONLY)
rdd.persist(StorageLevel.DISK_ONLY)
# Almacena el RDD únicamente en el disco. Esto es más lento que la memoria, pero asegura que los datos estén disponibles incluso con poca RAM.
# Resultado esperado: Los datos del RDD se guardarán en el disco tras la primera acción sobre este RDD.

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287

In [16]:
# Eliminar el RDD persistido del disco
rdd.unpersist()
# Libera el espacio en disco donde estaba almacenado el RDD persistido.
# Resultado esperado: El almacenamiento en disco utilizado por este RDD queda liberado.

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287

In [17]:
# Almacenar el RDD en memoria con el método cache()
rdd.cache()
# Es equivalente a `persist(StorageLevel.MEMORY_AND_DISK)`, por lo que primero intenta almacenar el RDD en memoria.
# Si la memoria es insuficiente, guarda los datos sobrantes en el disco.
# Resultado esperado: El RDD se almacena en memoria (y en disco si es necesario) tras la primera acción.

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287

In [18]:
# Eliminar el RDD almacenado en caché
rdd.unpersist()
# Libera el espacio en memoria y/o disco ocupado por el RDD cacheado.
# Resultado esperado: Se libera cual

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287

### **Resumen de operaciones:**
1. Se crea un RDD distribuido con los números del 0 al 9.
2. Se experimenta con diferentes niveles de persistencia (`MEMORY_ONLY`, `DISK_ONLY`, y `cache()`).
3. Cada vez que se almacena en caché o se persiste un RDD, se libera después usando `unpersist()` para liberar recursos.

Este código demuestra cómo manejar el almacenamiento en caché y persistencia de un RDD en Spark. Si necesitas ejemplos con acciones específicas (como `count()` o `collect()`), podemos añadirlos para observar los efectos del almacenamiento en caché. 😊